In [786]:
from sklearn import datasets, linear_model, metrics 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from mord import OrdinalRidge
from mord import LAD

In [787]:
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.layers import SpatialDropout1D
from keras.layers import Embedding
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [1247]:
def eval_confusion_matrix(actual, predicted):
    TN = 0
    FP = 0
    FN = 0
    TP = 0  
    for i in range(len(actual)):
        if actual[i]==0 or actual[i]==1:
            if actual[i]==predicted[i]:
                if actual[i]==1:
                    TP+=1
                elif actual[i]==0:
                    TN+=1
            else:
                if predicted[i]==1:
                    FP+=1
                elif predicted[i]==0:
                    FN+=1
    
    return TN, FP, FN, TP

def eval_recall(TP, FN):
    recall = TP/(TP+FN)
    return recall

def eval_acc(TN, FP, FN, TP):
    acc = (TN+TP)/(TN+FP+FN+TP)
    return acc

def eval_precision(TP, FP):
    if(TP+FP==0):
        return 0
    prec = TP/(TP+FP)
    return prec

def eval_f1(precision, recall):
    if(precision+recall==0):
        return 0
    f1 = 2*(precision*recall)/(precision+recall)
    return f1

def evaluate(actual, predicted):
    TN, FP, FN, TP = eval_confusion_matrix(actual, predicted)
    recall = eval_recall(TP, FN)
    prec = eval_precision(TP, FP)
    f1 = eval_f1(prec, recall)
    acc= eval_acc(TN, FP, FN, TP)
    print("Recall: ", recall)
    print("Precision: ", prec)
    print("F1 Score: ",f1)
    print("Accuracy: ", acc)
    return f1

In [1312]:
data = pd.read_csv("compiled_transcripts.csv", index_col='Participant_ID')

In [1313]:
data.head()

,Transcript,PHQ_Score,PHQ_Binary
Participant_ID,,,
300,good atlanta georgia um my parents are from he...,2,0
301,thank you mmm k i'm doing good thank you i'm f...,3,0
302,i'm fine how about yourself i'm from los ange...,4,0
303,okay how 'bout yourself here in california yea...,0,0
304,i'm doing good um from los angeles california ...,6,0


In [1314]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Los Angeles", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = [i.lower() for i in state_names]

In [1315]:
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    
    for state in state_names:
        text = text.replace(state, '')
    
    text = re.sub(r"i'll", 'i will', text)
    text = re.sub(r"i'm", 'i am', text)
    text = re.sub(r"it's", 'it is', text)
    text = re.sub(r"l_a", '', text)
    text = re.sub(r"u_s", '', text)
    text = re.sub(r"[^\w\s]", '', text) #remove everything except numbers and letters
    
    text = re.sub(r"dont", 'do not', text)
    text = re.sub(r"itll", 'it will', text)
    text = re.sub(r" um ", ' uh ', text)
    text = re.sub(r"couldve", 'could have', text)
    text = re.sub(r"yes", '', text)
    
    text = re.sub(r"synch", '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words) #remove stopwords
    return text

In [1316]:
data['Transcript'] = data['Transcript'].apply(clean_text)

In [1317]:
data.head()

,Transcript,PHQ_Score,PHQ_Binary
Participant_ID,,,
300,good atlanta uh parents uh love like weather l...,2,0
301,thank mmm k good thank oh great live west west...,3,0
302,fine part okay uh familys friends mixture peop...,4,0
303,okay bout yeah oh well big broad theres lot lo...,0,0
304,good uh uh cool weather beaches people schools...,6,0


In [1319]:
X = data['Transcript'].values
y = data['PHQ_Binary'].values

In [1321]:
def balance_data(X,y):
    new_y = []
    new_X = []
    dep_count = sum(y)
    count = 0
    for i in range(X.shape[0]):
        if y[i]==0:
            if count<dep_count:
                new_y.append(y[i])
                new_X.append(X[i])
                count+=1
        else:
            new_y.append(y[i])
            new_X.append(X[i])
    return np.array(new_X), np.array(new_y)

In [1322]:
X, y =balance_data(X,y)

In [1325]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index)+1

In [1326]:
max_length = 1000

In [1327]:
X = tokenizer.texts_to_sequences(X) # integer encode text
X = pad_sequences(X, maxlen=max_length, padding='post') # pad/truncate text sequences

In [1329]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=35)

In [1335]:
np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)

### Testing logistic regression and ridge classifier

In [1241]:
reg = linear_model.LogisticRegression()
# reg = linear_model.RidgeClassifierCV()

In [1242]:
reg.fit(X_train, y_train)

LogisticRegression()

In [1243]:
probas = reg.predict_proba(X_test)
print(probas)

[[9.98761604e-01 1.23839639e-03]
 [3.06131135e-01 6.93868865e-01]
 [9.99999968e-01 3.15245243e-08]
 [2.02803740e-02 9.79719626e-01]
 [4.94560216e-01 5.05439784e-01]
 [5.16187491e-01 4.83812509e-01]
 [7.48606265e-03 9.92513937e-01]
 [6.89223749e-01 3.10776251e-01]
 [1.27847844e-06 9.99998722e-01]
 [2.83999973e-05 9.99971600e-01]
 [8.90587854e-01 1.09412146e-01]]


In [1244]:
y_pred = reg.predict(X_test)

In [1245]:
y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0], dtype=int64)

In [1246]:
y_test

array([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

In [1248]:
evaluate(y_test, y_pred)

Recall:  0.8
Precision:  0.6666666666666666
F1 Score:  0.7272727272727272
Accuracy:  0.7272727272727273


0.7272727272727272

### Testing with Decision Tree

In [1249]:
from sklearn.tree import DecisionTreeClassifier

In [1273]:
def find_tree(upper_depth, upper_leaf):
    best_depth = 0
    best_leaf = 0
    best_f1 = 0
    for i in range(1, upper_depth+1):
        for j in range(1, upper_leaf+1):
            model = DecisionTreeClassifier(max_depth=i, min_samples_leaf=j) 
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            print("Max_depth: {} Min_samples_leaf: {}".format(i, j))
            f1 = evaluate(y_test, y_pred)
            if f1>best_f1:
                best_f1 = f1
                best_depth = i
                best_leaf = j
    return best_depth, best_leaf, best_f1

In [1274]:
best_depth, best_leaf, best_f1 = find_tree(100, 50)

Max_depth: 1 Min_samples_leaf: 1
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 2
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 3
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 4
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 5
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 6
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 7
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min_samples_leaf: 8
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Max_depth: 1 Min

Max_depth: 2 Min_samples_leaf: 41
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 2 Min_samples_leaf: 42
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 2 Min_samples_leaf: 43
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 2 Min_samples_leaf: 44
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 2 Min_samples_leaf: 45
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 2 Min_samples_leaf: 46
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 2 Min_samples_leaf: 47
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 2 Min_samples_leaf: 48
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 2 Min_samples_leaf: 49
Recall:  0.6
Precision:  1.0
F1 Score:  0.

Max_depth: 4 Min_samples_leaf: 27
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 4 Min_samples_leaf: 28
Recall:  0.8
Precision:  0.5
F1 Score:  0.6153846153846154
Accuracy:  0.5454545454545454
Max_depth: 4 Min_samples_leaf: 29
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 4 Min_samples_leaf: 30
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 4 Min_samples_leaf: 31
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 4 Min_samples_leaf: 32
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 4 Min_samples_leaf: 33
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 4 Min_samples_leaf: 34
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 4 M

F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 5 Min_samples_leaf: 45
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 5 Min_samples_leaf: 46
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 5 Min_samples_leaf: 47
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 5 Min_samples_leaf: 48
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 5 Min_samples_leaf: 49
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 5 Min_samples_leaf: 50
Recall:  1.0
Precision:  0.45454545454545453
F1 Score:  0.625
Accuracy:  0.45454545454545453
Max_depth: 6 Min_samples_leaf: 1
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 6 Min_samples_leaf: 2
Recall:  0.2
Precision:  0.3333333333333333
F1 Score:  0.25
Accuracy:  0.45454545454545453
Max_depth: 6 Min_samples_le

Max_depth: 7 Min_samples_leaf: 20
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 7 Min_samples_leaf: 21
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 7 Min_samples_leaf: 22
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 7 Min_samples_leaf: 23
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 7 Min_samples_leaf: 24
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 7 Min_samples_leaf: 25
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 7 Min_samples_leaf: 26
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 7 Min_samples_leaf: 27
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Ma

...

Max_depth: 94 Min_samples_leaf: 29
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 30
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 31
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 32
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 33
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 34
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 35
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 94 Min_samples_leaf: 36
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_dept

Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 3
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 4
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_leaf: 5
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 96 Min_samples_leaf: 6
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 96 Min_samples_leaf: 7
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 96 Min_samples_leaf: 8
Recall:  0.6
Precision:  0.75
F1 Score:  0.6666666666666665
Accuracy:  0.7272727272727273
Max_depth: 96 Min_samples_leaf: 9
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 96 Min_samples_leaf: 10
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 96 Min_samples_le

Max_depth: 97 Min_samples_leaf: 27
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 28
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 29
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 30
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 31
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 32
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 33
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_depth: 97 Min_samples_leaf: 34
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Max_

Max_depth: 98 Min_samples_leaf: 48
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 98 Min_samples_leaf: 49
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 98 Min_samples_leaf: 50
Recall:  1.0
Precision:  0.45454545454545453
F1 Score:  0.625
Accuracy:  0.45454545454545453
Max_depth: 99 Min_samples_leaf: 1
Recall:  0.4
Precision:  0.4
F1 Score:  0.4000000000000001
Accuracy:  0.45454545454545453
Max_depth: 99 Min_samples_leaf: 2
Recall:  0.2
Precision:  0.3333333333333333
F1 Score:  0.25
Accuracy:  0.45454545454545453
Max_depth: 99 Min_samples_leaf: 3
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 99 Min_samples_leaf: 4
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 99 Min_samples_leaf: 5
Recall:  0.6
Precision:  0.6
F1 Score:  0.6
Accuracy:  0.6363636363636364
Max_depth: 99 Min_samples_leaf: 6
Recall:  0.4
Precision:  0.5
F1 

Max_depth: 100 Min_samples_leaf: 21
Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454
Max_depth: 100 Min_samples_leaf: 22
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 100 Min_samples_leaf: 23
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 100 Min_samples_leaf: 24
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 100 Min_samples_leaf: 25
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 100 Min_samples_leaf: 26
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 100 Min_samples_leaf: 27
Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273
Max_depth: 100 Min_samples_leaf: 28
Recall:  0.8
Precision:  0.5
F1 Score:  0.6153846153846154
Accuracy:  0.54545

In [1275]:
print(best_depth, best_leaf)

2 18


In [1276]:
model = DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf=best_leaf) 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = evaluate(y_test, y_pred)

Recall:  1.0
Precision:  0.625
F1 Score:  0.7692307692307693
Accuracy:  0.7272727272727273


In [1277]:
print(y_pred)
print(y_test)

[0 0 1 1 1 1 1 1 1 1 0]
[0 0 1 1 1 0 0 0 1 1 0]


### Testing with kNN

In [1270]:
from sklearn.neighbors import KNeighborsClassifier

In [1271]:
def find_k(upper_k):
    best_k = 0
    best_f1 = 0
    for k in range(1,upper_k+1):
        neigh = KNeighborsClassifier(n_neighbors=k)
        neigh.fit(X_train, y_train)
        y_pred = neigh.predict(X_test)
        print("Value of k: ",k)
        f1 = evaluate(y_test, y_pred)
        if f1>best_f1:
            best_k = k
            best_f1 = f1
    return best_k, best_f1

In [1272]:
best_k, best_f1=find_k(10)

Value of k:  1
Recall:  0.0
Precision:  0
F1 Score:  0
Accuracy:  0.5454545454545454
Value of k:  2
Recall:  0.0
Precision:  0
F1 Score:  0
Accuracy:  0.5454545454545454
Value of k:  3
Recall:  0.0
Precision:  0
F1 Score:  0
Accuracy:  0.5454545454545454
Value of k:  4
Recall:  0.0
Precision:  0
F1 Score:  0
Accuracy:  0.5454545454545454
Value of k:  5
Recall:  0.4
Precision:  0.6666666666666666
F1 Score:  0.5
Accuracy:  0.6363636363636364
Value of k:  6
Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273
Value of k:  7
Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454
Value of k:  8
Recall:  0.2
Precision:  1.0
F1 Score:  0.33333333333333337
Accuracy:  0.6363636363636364
Value of k:  9
Recall:  0.2
Precision:  0.5
F1 Score:  0.28571428571428575
Accuracy:  0.5454545454545454
Value of k:  10
Recall:  0.0
Precision:  0
F1 Score:  0
Accuracy:  0.5454545454545454


In [1279]:
neigh = KNeighborsClassifier(n_neighbors=best_k)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.4
Precision:  1.0
F1 Score:  0.5714285714285715
Accuracy:  0.7272727272727273


0.5714285714285715

In [1280]:
print(y_pred)
print(y_test)

[0 0 0 1 0 0 0 0 1 0 0]
[0 0 1 1 1 0 0 0 1 1 0]


### Testing with gaussian naive bayes

In [1281]:
from sklearn.naive_bayes import GaussianNB

In [1282]:
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.6
Precision:  1.0
F1 Score:  0.7499999999999999
Accuracy:  0.8181818181818182


0.7499999999999999

In [1283]:
print(y_pred)
print(y_test)

[0 0 0 0 1 0 0 0 1 1 0]
[0 0 1 1 1 0 0 0 1 1 0]


### Testing with SVM

In [1285]:
from sklearn.svm import SVC

In [1286]:
model = SVC(C=10, gamma='auto', kernel='rbf')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  1.0
Precision:  0.45454545454545453
F1 Score:  0.625
Accuracy:  0.45454545454545453


0.625

### Testing with AdaBoost

In [1287]:
from sklearn.ensemble import AdaBoostClassifier

In [1294]:
model = AdaBoostClassifier(n_estimators=200, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.6
Precision:  0.5
F1 Score:  0.5454545454545454
Accuracy:  0.5454545454545454


0.5454545454545454

### Testing with Random forest

In [1289]:
from sklearn.ensemble import RandomForestClassifier

In [1309]:
model = RandomForestClassifier(n_estimators = 40)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
evaluate(y_test, y_pred)

Recall:  0.8
Precision:  0.8
F1 Score:  0.8000000000000002
Accuracy:  0.8181818181818182


0.8000000000000002

In [1310]:
print(y_pred)
print(y_test)

[0 1 1 0 1 0 0 0 1 1 0]
[0 0 1 1 1 0 0 0 1 1 0]


In [1311]:
model.predict_proba(X_test)

array([[0.5  , 0.5  ],
       [0.4  , 0.6  ],
       [0.35 , 0.65 ],
       [0.625, 0.375],
       [0.425, 0.575],
       [0.575, 0.425],
       [0.6  , 0.4  ],
       [0.6  , 0.4  ],
       [0.325, 0.675],
       [0.45 , 0.55 ],
       [0.525, 0.475]])

### Testing with Neural network

In [1208]:
y_train = to_categorical(y_train)

In [1209]:
max_features = vocab_size
maxlen = max_length
embedding_dim = 128

In [1210]:
model = Sequential()

model.add(Embedding(max_features, embedding_dim, input_shape=(maxlen,)))
model.add(Flatten())
# model.add(Dense(5, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [1211]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [1212]:
checkpoint_filepath = 'model1.h5'
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 10)
checkpoint =  ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [1213]:
history = model.fit(X_train, y_train, epochs=50, batch_size=1, validation_split=0.1, callbacks=[es, checkpoint])

C:\Users\65842\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 89 samples, validate on 10 samples
Epoch 1/50
89/89 [==============================] - ETA: 0s - loss: 0.9817 - accuracy: 0.58 - 1s 12ms/step - loss: 0.9834 - accuracy: 0.5730 - val_loss: 0.7268 - val_accuracy: 0.5000
Epoch 2/50
89/89 [==============================] - 1s 10ms/step - loss: 0.0395 - accuracy: 1.0000 - val_loss: 0.8011 - val_accuracy: 0.3000
Epoch 3/50
89/89 [==============================] - 1s 10ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.7346 - val_accuracy: 0.6000
Epoch 4/50
89/89 [==============================] - 1s 10ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.7422 - val_accuracy: 0.7000
Epoch 5/50
89/89 [==============================] - 1s 11ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.7438 - val_accuracy: 0.7000
Epoch 6/50
89/89 [==============================] - 1s 11ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.7438 - val_accuracy: 0.7000
Epoch 7/50
89/89 [==============================] - 1s 11ms/step - l

In [1214]:
model = load_model(checkpoint_filepath)

In [1215]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred)
print(y_test)

[0 1 1 1 0 0 1 0 0 0 0]
[0 0 1 1 1 0 0 0 1 1 0]


In [1216]:
evaluate(y_test, y_pred)

Recall:  0.4
Precision:  0.5
F1 Score:  0.4444444444444445
Accuracy:  0.5454545454545454


0.4444444444444445